# Plot 3D current line around Mars
We try to plot 3D current of Mars magnetosphere.

Author: Shi, Zhen (e-mail: Shi_Zhen@mail.iggcas.ac.cn)

Log:
2022/12/09 - initial version
2022/12/09 - upload to github

In [1]:
# import packages
import numpy as np
import scipy.io as scio
import pyvista as pv
import os
import sys

In [2]:
# parameters initalizing
Rm = 3.3935e6

In [3]:
# data information
data_folder = "D:\\program_sz\\python\\Mars_iono_current\\"
data_file = "current_MSE_global_1206.mat"


# -------------------------------------------------------------------------
# save information
sv_folder = "D:\\program_sz\\python\\Mars_iono_current\\pic\\"
key_str = data_file.split(".")[-2]
# sv_type = ".svg"
sv_type = ".png"

os.makedirs(sv_folder, exist_ok=True)

In [4]:
# load data 
dat_mat = scio.loadmat(data_folder+data_file)

# position 
x = dat_mat['X1']/Rm
y = dat_mat['Y1']/Rm
z = dat_mat['Z1']/Rm

x = np.swapaxes(x,0,1)
y = np.swapaxes(y,0,1)
z = np.swapaxes(z,0,1)

# current density
Jx = dat_mat['curlx']
Jy = dat_mat['curly']
Jz = dat_mat['curlz']

Jx = np.swapaxes(Jx,0,1)
Jy = np.swapaxes(Jy,0,1)
Jz = np.swapaxes(Jz,0,1)

In [5]:
# set grid
grid = pv.UniformGrid(dims = Jx.shape,
                        spacing = (0.2, 0.2, 0.2),
                        origin = (x.min(), z.min(), z.min()))

J_field = np.empty((len(Jx.flatten()), 3))
J_field[:, 0] = Jx.flatten('F')
J_field[:, 1] = Jy.flatten('F')
J_field[:, 2] = Jz.flatten('F')   
J_x = Jx.flatten('F')
grid['J'] = J_field 
grid['Jx'] = J_x 

In [6]:
grid.set_active_scalars('Jx', preference = 'point')

In [7]:
grid.point_data

pyvista DataSetAttributes
Association     : POINT
Active Scalars  : Jx
Active Vectors  : None
Active Texture  : None
Active Normals  : None
Contains arrays :
    J                       float64    (8000, 3)
    Jx                      float64    (8000,)              SCALARS

In [8]:
# set up plotter
pl = pv.Plotter(window_size=(2000, 1500), multi_samples=8, line_smoothing=True, polygon_smoothing=True)

pl.add_mesh(grid.outline(), color = 'k')

<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x0000025CFE5956D0) at 0x0000025C9E275460>

In [9]:
#  planet
sp = pv.Sphere(
    radius=1, center=(0, 0, 0), direction=(0, 0, 1), theta_resolution=120, phi_resolution=60
)

pl.add_mesh(sp, color = 'w')

<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x0000025CDDFDA210) at 0x0000025C9E2754C0>

In [10]:
# slice
slice1 = grid.slice(normal = 'x', origin = (0, 0, 0))
pl.add_mesh(slice1, scalars = 'Jx', cmap = 'seismic', clim = [-10, 10], opacity = 0.5)
# slice2 = grid.slice(normal = 'x', origin = (-1, 0, 0))
# pl.add_mesh(slice2, scalars = 'Jx', cmap = 'seismic', clim = [-10, 10], opacity = 0.5)
# slice3 = grid.slice(normal = 'x', origin = (1, 0, 0))
# pl.add_mesh(slice3, scalars = 'Jx', cmap = 'seismic', clim = [-10, 10], opacity = 0.5)


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x0000025CFE74E010) at 0x0000025C9E275BE0>

In [11]:
slice

slice

In [12]:
# set up series of seeds for streamlines

# the postion of each seed
sn = 10
s1 = np.ones(sn)
sr = 1.4
sth = np.linspace(0, 2*np.pi, sn, endpoint = False)
sz = 0*s1
sx = sr*np.cos(np.linspace(0, 2*np.pi, sn, endpoint = False))
sy = sr*np.sin(np.linspace(0, 2*np.pi, sn, endpoint = False))

# the parameters of each seed
seed_r = 0.05
seed_thr = 60
seed_phr = 30
seed_c = 'm'


# -------------------------------------------------------------------------
# some parameters about the streamlines
sl_c = 'c'


In [13]:
# plot streamlines

# cone for streamlines
cn = pv.Cone(resolution = 20)

for xi, yi, zi in zip(sx, sy, sz):
    # create seed
    tmp_seed = pv.Sphere(radius = seed_r, center = (xi, yi, zi), direction = (0, 0, 1),
    theta_resolution = seed_thr, phi_resolution = seed_phr)

    # create streamline
    tmp_sl = grid.streamlines(
        vectors = 'J',
        start_position= (xi, yi, zi),
        integration_direction = 'both',
        initial_step_length=0.15,
        min_step_length=0.01, 
        max_step_length=0.5, 
        max_steps=2000,
        terminal_speed=1e-12, 
        max_error=1e-06,
        max_time=50
    )

    # add direction cones
    sl_cn = tmp_sl.glyph(orient='J', scale=False, factor=0.08, geom=cn, tolerance = 0.12)

    # add those to the plotter
    pl.add_mesh(tmp_seed, color = seed_c)
    pl.add_mesh(tmp_sl.tube(radius = 0.01), color = sl_c)
    pl.add_mesh(sl_cn, color = sl_c)
    

In [14]:
# show
pl.show_axes()
pl.show()

ViewInteractiveWidget(height=1500, layout=Layout(height='auto', width='100%'), width=2000)

In [15]:
# camera postion
pl.camera_position

[(7.388224244530522, 7.390758753235661, 7.383492827828252),
 (-0.00253450870513916, 0.0, -0.007265925407409668),
 (0.0, 0.0, 1.0)]